In [64]:
import datetime
import numpy as np
import pandas as pd

In [44]:
# Read the data from the tab separated file
df_raw = pd.read_csv('demo_r_mweek3.tsv', sep='\t')

# Get the name of the comma separated column which needs to be expanded by splitting on the comma's
col_to_split = df_raw.columns[0]

# Expand that column and remove the old one
df_raw[['unit', 'sex', 'age', 'country']] = df_raw[col_to_split].str.split(",", expand=True)
df_raw.drop(col_to_split, axis=1, inplace=True)

# Only select the rows where the sex is 'T' (so the sum of Male and Female)
df_raw = df_raw[(df_raw['sex'] == 'T') & (df_raw['age'] == 'TOTAL')]

In [45]:
# Only select the rows of countries, not of provinces or smaller
mask = df_raw['country'].str.len() == 2
df_raw = df_raw.loc[mask]

# Optional: Add a column with the country name, rather than the country abbreviation
# country_dict = {'AM': 'Armenia', 'AT': 'Austria', 'BE': 'Belgium', 'BG': 'Bulgaria', 'CH': 'Switzerland',
#                 'CZ': 'Czech Republic', 'DK': 'Denmark', 'EE': 'Estonia', 'ES': 'Spain', 'FI': 'Finland',
#                 'GE': 'Georgia', 'IS': 'Iceland', 'LI': 'Liechtenstein', 'LT': 'Lithuania', 'LU': 'Luxembourg',
#                 'LV': 'Latvia', 'NL': 'Netherlands', 'PT': 'Portugal', 'RS': 'Serbia', 'SE': 'Sweden', 'SK': 'Slovakia'}
# df_raw['country_name'] = df_raw['country'].replace(country_dict)

# Drop the columns where only one values was selected for
df_raw.drop(['unit', 'sex', 'age'], axis=1, inplace=True)

# Replace all ': ' values in the dataframe with np.nan values
df_raw.replace(': ', np.nan, inplace=True)

# Remove all ' p' parts from the number of deaths (provisional numbers) Note: ' p' is removed from _all_ columns
df_raw.replace(' p', '', regex=True, inplace=True)

In [46]:
# Create a separate country dataframe to merge later with the stacked week values
countries = df_raw[['country']]
df_raw.drop('country', axis=1, inplace=True)

In [61]:
# Stack the week values and make a dataframe with a single index from this
df = df_raw.stack(dropna=False)
df = df.reset_index(level=[0, 1])

# Add the counties to this dataframe
df = df.merge(countries, left_on='level_0', right_index=True)

# Rename the week column and the deaths column
df.rename({'level_1': 'week', 0: 'deaths'}, axis=1, inplace=True)

# Drop the level_0 column (the original index)
df.drop('level_0', axis=1, inplace=True)

# Make sure the deaths column is of type float (to keep the np.NaN it cannot be int)
df.deaths = df.deaths.astype(float)

# Add source column and add the source to all rows
df['source'] = 'https://data.europa.eu/euodp/en/data/dataset/WHum2Ir8F4KYmrrkj1sRQ'

# Add filter and notes columns and leave them empty
df['filter'] = ''
df['notes'] = ''

In [82]:
# Add the first day of the week as a column
df['first_day'] = pd.to_datetime(df['week'] + '-1', format="%GW%V -%u")  # The -1 is for the first day of the week

# Add the last day of the week as a column
df['last_day'] = pd.to_datetime(df['week'] + '-7', format="%GW%V -%u")  # The -7 is for the first day of the week

# TODO Remove possible weird weeks, spanning over multiple years